In [159]:
import requests, zipfile, io
import pandas as pd

years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
month_nums = ["01","02","03","04","05","06","07","08","09","10","11","12"]
years_extra = ["2013","2014","2015","2016","2017"]
last_years = ["2018","2019","2020"]
dfs_dates = []
dfs = []

for year in years:
    r = requests.get(f"https://www.bart.gov/sites/default/files/docs/ridership_{year}.zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall('c:\\Users\\carso\\OneDrive - UW-Madison\\23_24_Fall\\econ695\\challenge_problems\\ridership')

    if year in years_extra:
        for month in months:
            file = f"ridership/ridership_{year}/Ridership_{month}{year}.xlsx"
            df = pd.read_excel(file, sheet_name='Weekday OD', header=1, index_col=0, nrows=43, usecols=range(44)).fillna('')
            dfs_dates.append(f"{month}_{year}")
            dfs.append(df)
    elif year in last_years:
        for month in month_nums:
            file = f"ridership/ridership_{year}/Ridership_{year}{month}.xlsx"
            if year == '2019' or year == '2020':
                df = pd.read_excel(file, sheet_name='Avg Weekday OD', header=1, index_col=0, nrows=43, usecols=range(44)).fillna('')
                dfs_dates.append(f"{month}_{year}")
                dfs.append(df)
            else:
                if month == "01":
                    df = pd.read_excel(file, sheet_name='Weekday OD', header=1, index_col=0, nrows=43, usecols=range(44)).fillna('')
                    dfs_dates.append(f"{month}_{year}")
                    dfs.append(df)
                else:
                    df = pd.read_excel(file, sheet_name='Avg Weekday OD', header=1, index_col=0, nrows=43, usecols=range(44)).fillna('')
                    dfs_dates.append(f"{month}_{year}")
                    dfs.append(df)
    else:
        for month in months:
            file = f"ridership/Ridership_{month}{year}.xlsx"
            df = pd.read_excel(file, sheet_name='Weekday OD', header=1, index_col=0, nrows=43, usecols=range(44)).fillna('')
            dfs_dates.append(f"{month}_{year}")
            dfs.append(df)

trans_dfs = []
for df in dfs:
    stacked_df = df.stack()
    stacked_df = stacked_df.reset_index()
    stacked_df.columns = ['row_value', 'col_name', 'value']
    stacked_df['index_name'] = stacked_df['row_value'].astype(str) + '-' + stacked_df['col_name'].astype(str)
    result_df = stacked_df.set_index('index_name')[['value']]
    result_df = result_df.rename(columns={"value": "year"})
    trans_dfs.append(result_df)

compiled_df = pd.concat(trans_dfs, axis=1, join='inner')
compiled_df.columns = dfs_dates
compiled_df

,January_2010,February_2010,March_2010,April_2010,May_2010,June_2010,July_2010,August_2010,September_2010,October_2010,...,03_2020,04_2020,05_2020,06_2020,07_2020,08_2020,09_2020,10_2020,11_2020,12_2020
index_name,,,,,,,,,,,,,,,,,,,,,
RM-RM,4.894737,5.052632,7.391304,7.409091,6.90,7.227273,7.095238,4.954545,6.000000,6.45,...,12.590909,6.181818,6.30,7.772727,7.454545,8.761905,8.476190,9.571429,9.166667,6.10
RM-EN,107.526316,115.263158,115.478261,117.454545,119.15,118.636364,120.714286,115.227273,117.904762,120.40,...,74.818182,26.363636,25.10,35.136364,29.318182,26.857143,26.523810,29.571429,30.166667,27.95
RM-EP,108.157895,108.526316,106.304348,106.681818,110.45,109.227273,97.904762,99.545455,114.000000,111.75,...,50.863636,16.954545,17.75,24.409091,26.681818,22.523810,22.809524,29.142857,28.444444,24.90
RM-NB,89.684211,94.263158,90.304348,77.727273,95.60,72.500000,72.095238,78.227273,93.904762,86.40,...,41.727273,17.772727,18.00,22.863636,23.363636,21.857143,21.809524,22.428571,22.777778,21.20
RM-BK,345.526316,398.000000,378.956522,381.136364,369.10,374.909091,373.666667,336.772727,399.714286,412.05,...,215.772727,39.090909,50.45,65.590909,77.454545,84.714286,81.904762,81.571429,81.500000,68.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-WP,31.157895,29.210526,25.391304,25.500000,26.35,28.000000,28.428571,27.818182,25.904762,26.15,...,5.545455,0.545455,1.95,1.818182,2.227273,2.095238,2.380952,1.666667,2.222222,3.10
MB-SS,49.684211,54.052632,52.869565,52.954545,56.90,54.136364,59.904762,59.590909,61.142857,59.65,...,30.454545,3.181818,4.15,6.090909,6.227273,5.047619,5.000000,6.571429,5.444444,5.30
MB-SB,46.157895,52.421053,51.869565,52.454545,51.10,61.000000,62.095238,66.681818,66.619048,65.35,...,16.909091,3.818182,3.45,3.909091,5.636364,6.619048,7.809524,6.380952,4.944444,3.55


In [157]:
compiled_df.to_csv('file.csv')